In [1]:
import torch
from diffusers import AutoencoderKL
from transformers import CLIPTextModel, CLIPTokenizer
from pathlib import Path

/home/tran.duc.trungb/miniconda3/envs/triton-huggingface/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import json

import numpy as np
import torch
from diffusers import LMSDiscreteScheduler, UNet2DConditionModel
from torch.utils.dlpack import from_dlpack, to_dlpack
from tqdm.auto import tqdm
from transformers import CLIPTokenizer

/home/tran.duc.trungb/miniconda3/envs/triton-huggingface/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
unet = UNet2DConditionModel.from_pretrained(
            "CompVis/stable-diffusion-v1-4",
            subfolder="unet",
            revision="fp16",
            torch_dtype=torch.float16,
            use_auth_token=True,
        ).to("cuda")

Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.
unet/config.json: 100%|██████████| 772/772 [00:00<00:00, 2.21MB/s]
/home/tran.duc.trungb/miniconda3/envs/triton-huggingface/lib/python3.10/site-packages/diffusers/utils/hub_utils.py:336: FutureWarning: You are loading the variant fp16 from CompVis/stable-diffusion-v1-4 via `revision='fp16'`. This behavior is deprecated and will be removed in diffusers v1. One should use `variant='fp16'` instead. However, it appears that CompVis/stable-diffusion-v1-4 currently does not have a diffusion_pytorch_model.fp16.safetensors file in the 'main' branch of CompVis/stable-diffusion-v1-4. 
 The Diffusers team and community would be very grateful if you could open an issue: https://github.com/huggingface/d

In [2]:

prompt = "Draw a dog"
vae = AutoencoderKL.from_pretrained(
    "CompVis/stable-diffusion-v1-4", subfolder="vae", use_auth_token=True
)

tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-large-patch14")
text_encoder = CLIPTextModel.from_pretrained("openai/clip-vit-large-patch14")

vae.forward = vae.decode

Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.


In [3]:
ONNX_PATH = Path("../triton_workspace/onnx_models")

ONNX_PATH.mkdir(exist_ok=True)

In [4]:

torch.onnx.export(
    vae,
    (torch.randn(1, 4, 64, 64), False),
    (ONNX_PATH / "vae.onnx").as_posix(),
    input_names=["latent_sample", "return_dict"],
    output_names=["sample"],
    dynamic_axes={
        "latent_sample": {0: "batch", 1: "channels", 2: "height", 3: "width"},
    },
    do_constant_folding=True,
    opset_version=14,
)

/home/tran.duc.trungb/miniconda3/envs/triton-huggingface/lib/python3.10/site-packages/diffusers/models/upsampling.py:149: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert hidden_states.shape[1] == self.channels
/home/tran.duc.trungb/miniconda3/envs/triton-huggingface/lib/python3.10/site-packages/diffusers/models/upsampling.py:165: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if hidden_states.shape[0] >= 64:
/home/tran.duc.trungb/miniconda3/envs/triton-huggingface/lib/python3.10/site-packages/diffusers/models/autoencoders/autoencoder_kl.py:306: Tra

In [5]:

text_input = tokenizer(
    prompt,
    padding="max_length",
    max_length=tokenizer.model_max_length,
    truncation=True,
    return_tensors="pt",
)

torch.onnx.export(
    text_encoder,
    (text_input.input_ids.to(torch.int32)),
    (ONNX_PATH / "text_encoder.onnx").as_posix(),
    input_names=["input_ids"],
    output_names=["last_hidden_state", "pooler_output"],
    dynamic_axes={
        "input_ids": {0: "batch", 1: "sequence"},
    },
    opset_version=14,
    do_constant_folding=True,
)

/home/tran.duc.trungb/miniconda3/envs/triton-huggingface/lib/python3.10/site-packages/transformers/modeling_attn_mask_utils.py:86: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if input_shape[-1] > 1 or self.sliding_window is not None:
/home/tran.duc.trungb/miniconda3/envs/triton-huggingface/lib/python3.10/site-packages/transformers/modeling_attn_mask_utils.py:162: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if past_key_values_length > 0:
/home/tran.duc.trungb/miniconda3/envs/triton-huggingface/lib/python3.10/site-packages/transformers/models/clip/mo